# MLP
Test of simple MLP models with different amount of neurons, hidden layers and size of input vector
## Load Data

In [40]:
import torch
from torch.utils.data import Dataset, DataLoader, random_split
import torch.nn as nn
 
import pandas as pd
import numpy as np

from tqdm import tqdm

import plotly.express as px
from torch.optim import Adam


import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler

In [41]:
df = pd.read_csv("../../dataset_generation/test_datasets/data_time.csv")
df_1 = df[['t', 'X', 'Y', 'Z']]
df_diff = pd.DataFrame(df_1.iloc[1:].values - df_1.iloc[:-1].values, columns=["dt", "dx", "dy", "dz"])

std = StandardScaler()
df_diff[['dt']] = std.fit_transform(df_diff[['dt']])

In [42]:
class SequentDataset(Dataset):
    def __init__(self, dataframe: pd.DataFrame, n_dots=1, n_dot_parameters=4):
        self.n_dot_parameters = n_dot_parameters
        self.n_dots = n_dots
        all_data = self.__make_stack(dataframe)
        self.X_ = all_data[:, :-n_dot_parameters]
        self.y_ = all_data[:, -n_dot_parameters:]
        
    def __len__(self):
        return len(self.X_)
    
    def __getitem__(self, idx):
        return self.X_[idx], self.y_[idx]
    
    def __make_stack(self, df: pd.DataFrame):
        stacks = [df.iloc[:-self.n_dots]] + [df.iloc[i:].values if (self.n_dots == i) else df.iloc[i:-(self.n_dots - i)].values  for i in range(1, self.n_dots + 1)]
        return torch.tensor(np.hstack(stacks), dtype=torch.float32)
    
    def plotData(self, i = 0, f = -1):
        fig = px.line_3d(x = self.X_[i:f, 1], y = self.X_[i:f, 2], z = self.X_[i:f, 3])
        fig.show()

In [43]:
N_DOTS = 100
N_DOT_PARAMETERS = 4

In [44]:
dataset = SequentDataset(df_diff[0:5000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
test = SequentDataset(df_diff[5000:6000], n_dots=N_DOTS, n_dot_parameters=N_DOT_PARAMETERS)
dataset.plotData()

In [45]:
train_data, val_data = random_split(dataset,[0.8,0.2])

train_loader = DataLoader(train_data, batch_size=8, shuffle=True)
val_loader = DataLoader(val_data, batch_size=8, shuffle=False)

## Create Model

In [46]:
class MLPModel(nn.Module):
    def __init__(self, input_dots_amount, input_dot_parameters_amount, output_dots_amount, output_dot_parameters_amount, hidden_layers_lengths, dropout_probability=0, activation=nn.ReLU()):
        super(MLPModel, self).__init__()
        self.layers = nn.ModuleList()
        layer_lengths = [input_dots_amount * input_dot_parameters_amount] + hidden_layers_lengths + [output_dots_amount * output_dot_parameters_amount]
        self.dropout = nn.Dropout(dropout_probability)
        self.activation = activation
        for i in range(1, len(layer_lengths)):
            self.layers.append(nn.Linear(layer_lengths[i - 1], layer_lengths[i]))
    
    
    def forward(self, X: torch.Tensor):
        X_0 = X.clone()
        for layer in self.layers[:-1]:
            X = layer(X)
            X = self.activation(X)
            X = self.dropout(X)
        return self.layers[-1](X)

In [47]:
model = MLPModel(input_dots_amount=N_DOTS, input_dot_parameters_amount=N_DOT_PARAMETERS, output_dots_amount=1, output_dot_parameters_amount=4, hidden_layers_lengths=[100, 70, 50])
loss_model = nn.MSELoss(reduction='mean')
opt = Adam(model.parameters(), lr=0.001)

In [48]:
EPOCH = 250
train_losses = []
val_losses = []

for epoch in range(EPOCH):

    # Обучение
    model.train()
    # train_loop = tqdm(train_loader,leave=False)
    train_loss = []
    for X, y in train_loader:
        pred = model(X)
        loss = loss_model(pred, y)
        train_loss.append(loss.item())

        opt.zero_grad()
        loss.backward()

        opt.step()
        mean_train_loss = sum(train_loss)/len(train_loss)
        # train_loop.set_description(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}")

    train_losses.append(mean_train_loss)
    
    # Валидация
    model.eval()
    with torch.no_grad():
        val_loss = []
        for X, y in val_loader:
            pred = model(X)
            loss = loss_model(pred, y)
            val_loss.append(loss.item())

        mean_val_loss = sum(val_loss)/len(val_loss)
        val_losses.append(mean_val_loss)

    print(f"Epoch [{epoch+1}/{EPOCH}], train_loss = {mean_train_loss:.4f}, val_loss = {mean_val_loss:.4f}")

Epoch [1/250], train_loss = 1.1017, val_loss = 0.3894
Epoch [2/250], train_loss = 0.4561, val_loss = 0.3604
Epoch [3/250], train_loss = 0.3700, val_loss = 0.2941
Epoch [4/250], train_loss = 0.3387, val_loss = 0.2723
Epoch [5/250], train_loss = 0.3292, val_loss = 0.2978
Epoch [6/250], train_loss = 0.3250, val_loss = 0.2933
Epoch [7/250], train_loss = 0.2942, val_loss = 0.2751
Epoch [8/250], train_loss = 0.2801, val_loss = 0.2275
Epoch [9/250], train_loss = 0.2833, val_loss = 0.3563
Epoch [10/250], train_loss = 0.2654, val_loss = 0.2628
Epoch [11/250], train_loss = 0.2697, val_loss = 0.2568
Epoch [12/250], train_loss = 0.2492, val_loss = 0.2716
Epoch [13/250], train_loss = 0.2550, val_loss = 0.2427


KeyboardInterrupt: 

In [36]:
start, _  = dataset[0]
predictions = []
model.eval()

for i in range(1,500):
    with torch.no_grad():
        pred = model(start)
        predictions.append(pred)
        start = torch.hstack([start[N_DOT_PARAMETERS:], pred]) 
        # start = pred   

In [37]:
p_x = pd.DataFrame(map(lambda t: t.detach().tolist(), predictions), columns=["Time", "X", "Y", "Z"])
_, y  = dataset[:500]
p_y = pd.DataFrame(y.detach().numpy(), columns=["Time", "X", "Y", "Z"])

p_x = p_x.cumsum()
p_y = p_y.cumsum()

In [38]:
p_x["C"] = "Predict"
p_y["C"] = "True"

d = pd.concat([p_x,p_y],axis=0)

In [39]:
fig = px.line_3d(d, x="X", y="Y", z="Z", color='C')
# fig = px.scatter_3d(data, x="X", y="Y", z="Z")
fig.show()